## Initiatlize the Libraries

In [3]:
import os
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display






In [6]:

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"

openai = OpenAI(api_key=openai_api_key)
gemini_client = OpenAI(api_key=google_api_key, base_url=gemini_url)
deepseek_client = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
groq_client = OpenAI(api_key=groq_api_key, base_url=groq_url)



In [20]:
def record(model, reply):
    display(Markdown(f"## Model {model} \n\n Response {reply} \n\n ## Number of words {len(reply.split(' '))}" )  )            

In [ ]:
system_message = """
You are a helpful assistant that can answer questions and help with tasks.
Do not provide any additional text outside of the ccontext of the request
"""

user_message = """
Explain quantum computing in simple terms to a 8th grade student
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

model="gpt-4o-mini"

response = openai.chat.completions.create(
    model=model,
    messages=messages,
)

reply = response.choices[0].message.content
record(model, reply)

In [ ]:
## Let us create a chat between two chatbots.
## one from open AI and another from deepseek

gpt_model = "gpt-4o-mini"
deepseek_model = "deepseek-chat"

gpt_system_prompt = """
You are a good debater and is very argumentattive in nature. 
You try to make that you have the final say in everything. 
You are not unreasonable in your argument but will see everything with a critical eye
keep your response short and concise. Keep it less than 100 words.
"""

deepseek_system_prompt = """
You are a person with positive frame of mind. 
You are not argumentative and try to be neutral in your approach. 
You are not a debater and try to find common ground with the other person. 
You are a person with a positive frame of mind. 
keep your response short and concise. Keep it less than 100 words.
"""

gpt_messages = ["Hello, how are you?"]
deepseek_messages = ["Hi"]

def call_gpt():
    messages = [ {"role": "system", "content": gpt_system_prompt}]
    for gpt, deepseek in zip(gpt_messages, deepseek_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": deepseek})
        
    response = openai.chat.completions.create(model=gpt_model, messages=messages)
    reply = response.choices[0].message.content
    return reply

def call_deepseek():
    messages = [ {"role": "system", "content": deepseek_system_prompt}]
    for deepseek, gpt in zip(deepseek_messages, gpt_messages):
        messages.append({"role": "user", "content": deepseek})
        messages.append({"role": "assistant", "content": gpt})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    response = deepseek_client.chat.completions.create(model=deepseek_model, messages=messages)
    reply = response.choices[0].message.content
    return reply

print(f"GPT: {gpt_messages[0]} \n\n Deepseek: {deepseek_messages[0]}")

for i in range(5):
    gpt_reply = call_gpt()
    print(f"GPT: {gpt_reply}")
    gpt_messages.append(gpt_reply)

    deepseek_reply = call_deepseek()
    print(f"Deepseek: {deepseek_reply}\n\n")
    deepseek_messages.append(deepseek_reply)



In [ ]:
## Let us create simple chat UI using gradio
## Use space for Named entity extraction of cities in a sentence
## You have to do "uv add spacy"
## You also have to install the model using "uv run --with spacy spacy download en_core_web_sm"

import gradio as gr
import base64
from PIL import Image
from io import BytesIO
import spacy



ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}

model = "gpt-4o-mini"

print(system_message)

def update_system_message(destination_city, system_message):
    if destination_city in ticket_prices:
        system_message = system_message + f"The ticket price to {destination_city} is {ticket_prices.get(destination_city)}"
    return system_message

def chat(history):

    system_message = "You are a helpful airline agent. You are given a question and you need to answer it." 
    system_message += "You are not allowed to say that you are an AI assistant."
    system_message += "If you do not know the answer, just say you don't know"

    message = history[-1]["content"]
    destination_city = extract_city(message)
    system_message = update_system_message(destination_city.text.strip.lower(), system_message) if destination_city else system_message
    image = create_image(destination_city.text) if destination_city else None
    messages = [ {"role": "system", "content": system_message}]
    messages.extend(history)
    messages.extend([{"role": "user", "content": message}])
    response = openai.chat.completions.create(model=model, messages=messages)
    reply = response.choices[0].message.content
    history.extend([{"role": "assistant", "content": reply}])
    return history, image

def create_image(city):
    response = openai.images.generate(model="dall-e-3", 
                                      prompt=f"An image representing the culture and tourist attractions of the city of {city}, in a vbrant pop-art style",
                                      size = "1024x1024",
                                      n=1,
                                      response_format="b64_json")
    image_base64 = response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    return image

nlp = spacy.load('en_core_web_sm')

def extract_city(message):
    doc = nlp(message)
    city = doc.ents[0] if len(doc.ents) > 0 else None
    return city



## Now setup gradio UI

with gr.Blocks() as UI:
    gr.Markdown("Airline Agent")
    with gr.Row():
       chatbot = gr.Chatbot(height=400, type="messages")
       image_output = gr.Image(height=400)
    with gr.Row():
        user_input = gr.Textbox(placeholder="Chat with your Travel Assistant", label="User Input")

    def user_input_handler(message, history):
        history += [{"role": "user", "content": message}]
        print(history)
        return "", history

    user_input.submit(user_input_handler, 
                      inputs=[user_input, chatbot], 
                      outputs=[user_input, chatbot]
                      ).then(chat, 
                             inputs=chatbot, 
                             outputs=[chatbot, image_output])

UI.launch(inbrowser=True)



In [52]:
## Setup tool calling from LLM

## Step-1 Create the function
def get_ticket_price(destination_city):
    return ticket_prices.get(destination_city.strip().lower(),"unknown")

## Step-2 Define metadata of the function for LLM to understand. This is nothing but setting up the tool

get_price = {
    "name": "get_ticket_price",
    "description": "Get the ticket price for a destination city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {"type": "string", "description": "The destination city"}
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

## Step-3 assemble all tools to be made available to the LLM
tools = [{"type": "function", "function": get_price}]

## Step-4 Create the LLM client and make a call to LLM
openai_client = OpenAI(api_key=openai_api_key) 

system_message = "You are a helpful airline agent. You are given a question and you need to answer it." 
system_message += "You are not allowed to say that you are an AI assistant."
system_message += "If you do not know the answer, just say you don't know"

messages = [{"role": "system", "content": system_message}]
messages.append({"role": "user", "content": "What is the ticket price for Tokyo?"})

response = openai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools
)

## Step-5 Define a function on how to handle tool call

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    if tool_name == "get_ticket_price":
        city = tool_args.get("destination_city","unknown")
        price = get_ticket_price(city)
    
    response = {
        "role": "tool",
        "content": f"The ticket price for {tool_args.get('destination_city')} is {price}",
        "tool_call_id": tool_call.id
    }
    return response,city
    
def get_ticket_price(destination_city):
    return ticket_prices.get(destination_city.strip().lower(),"unknown")

#Step-6 Handle if the response of LLM finishes with a tool_call

if response.choices[0].finish_reason == "tool_calls":
    message = response.choices[0].message
    response,city = handle_tool_call(message)
    messages.append(message)
    messages.append(response)
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )
print(response.choices[0].message.content)



The ticket price for Tokyo is $1400.


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

text = "I want to go from New Delhi to Mumbai"

doc = nlp(text)

for entity in doc.ents:
   if entity.label_ == 'GPE':
      print(entity.text)

In [77]:
import duckdb as dd

conn = dd.connect("ticket_price.db")
conn.execute("CREATE or REPLACE TABLE ticket_prices (city VARCHAR, price INT)")
conn.execute("INSERT INTO ticket_prices VALUES ('London', 100), ('Paris', 200), ('Tokyo', 300), ('Sydney', 400), ('Mumbai',500)")
result = conn.execute(f"SELECT price FROM ticket_prices WHERE city = '{city}'").df()
print(result['price'].iloc[0])
conn.close()

300


[{'role': 'user', 'metadata': None, 'content': 'Hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'I would like to go to Mumbai', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Great choice! Could you please provide me with the departure city or airport, so I can help you with ticket information?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'IAD', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'm sorry, but I couldn't find the ticket price for a flight from IAD to Mumbai at this moment. If you have any other questions or need assistance with something else, feel free to ask!", 'options': None}, {'role': 'user', 'content': 'Can you check again'}]
{'destination_city': 'Mumbai', 'price': 500}


In [ ]:
## Retrieve information from DB to set the right context for LLM using function call.

import duckdb as dd


def get_ticket_price(city):
    conn = dd.connect("ticket_price.db")
    result = conn.execute(f"SELECT price FROM ticket_prices WHERE city = '{city}'").df()
    if result.empty:
        response = {"destination_city": city, "price": "unknown"}
    else:
        response = {"destination_city": city, "price": int(result['price'].iloc[0])}
    print(response)
    return response


price_function = {"name": "get_ticket_price", 
                  "description": "Get the ticket price for a destination city", 
                  "parameters": {
                      "type": "object",
                      "properties": {
                          "destination_city": {"type": "string", "description": "The destination city"},
                      },
                      "required": ["destination_city"],
                      "additionalProperties": False
                  }
                  } 

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    city = tool_args.get("destination_city")
    if tool_name == "get_ticket_price":
       response = {"role": "tool", 
                   "content": json.dumps(get_ticket_price(city)),
                   "tool_call_id": tool_call.id
                   }
    else:
        response = {"role": "tool", 
                    "content": json.dumps({"destination_city":"unknown", "price": "unknown"}),
                    "tool_call_id": tool_call.id 
                    }
    return response, city


tools = [{"type": "function", "function": price_function}]

openai_client = OpenAI(api_key=openai_api_key)
model = "gpt-4o-mini"

def chat(history):
    system_message = "You are a helpful airline agent. You are given a question and you need to answer it." 
    system_message += "You are not allowed to say that you are an AI assistant."
    system_message += "If you do not know the answer, just say you don't know"

    messages = [{"role": "system", "content": system_message}]
    messages.extend(history)
    message = history[-1]["content"]
    
    #destination_city = extract_city(message)

    #image = create_image(destination_city) if destination_city != 'unknown' else None    
    image = None

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response,city = handle_tool_call(message)
        image = create_image(city) 
        messages.append(message)
        messages.append(response)
        response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
    history += [{"role": "assistant", "content": response.choices[0].message.content}]
    return history, image



# Now setup gradio UI

with gr.Blocks() as UI:
    gr.Markdown("Airline Agent")
    with gr.Row():
       chatbot = gr.Chatbot(height=400, type="messages")
       image_output = gr.Image(height=400)
    with gr.Row():
        user_input = gr.Textbox(placeholder="Chat with your Travel Assistant", label="User Input")

    def user_input_handler(message, history):
        history += [{"role": "user", "content": message}]
        print(history)
        return "", history

    user_input.submit(user_input_handler, 
                      inputs=[user_input, chatbot], 
                      outputs=[user_input, chatbot]
                      ).then(chat, 
                             inputs=chatbot, 
                             outputs=[chatbot, image_output])

UI.launch(inbrowser=True)


